# Wikipedia Notable Life Expectancies
# [Notebook  : Data Cleaning Part ](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean10_thanak_2022_08_01.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [3]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean9.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean9", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98057 rows and 38 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None,None,2.564949,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

In [4]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98055,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",None,,MNA,None,None,None,None,None,None,None,None,50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,0,0,0,0,0,1,0,0,1,0,0,0,2
98056,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,,member of the Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None,None,1.386294,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

In [5]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
57744,4,Ranjan Baindoor,", 66, Indian cricketer.",https://en.wikipedia.org/wiki/Ranjan_Baindoor,3,2016,April,None,None,,None,None,None,None,None,None,None,None,None,66.0,None,India,None,None,1.386294,0,0,0,0,0,0,1,0,0,0,0,0,1
81579,11,Thorkild Grosbøll,", 72, Danish Lutheran clergyman.",https://en.wikipedia.org/wiki/Thorkild_Grosb%C3%B8ll,8,2020,May,None,None,Lutheran clergyman,None,None,None,None,None,None,None,None,None,72.0,None,Denmark,None,None,2.197225,0,0,0,0,0,0,0,0,0,0,0,0,0
6791,15,Karsten Andersen,", 77, Norwegian conductor.",https://en.wikipedia.org/wiki/Karsten_Andersen,6,1997,December,None,None,,None,None,None,None,None,None,None,None,None,77.0,None,Norway,None,None,1.945910,0,0,0,0,0,1,0,0,0,0,0,0,1
61565,8,Abdulkadir Kure,", 60, Nigerian politician, Governor of Niger State .",https://en.wikipedia.org/wiki/Abdulkadir_Kure,6,2017,January,,None,,Governor of State,None,None,None,None,None,None,None,None,60.0,None,Nigeria,None,1999 2007,1.945910,0,0,0,0,0,0,0,0,1,0,0,0,1
95608,24,Stevan K. Pavlowitch,", 88, Serbian-born British historian.",https://en.wikipedia.org/wiki/Stevan_K._Pavlowitch,9,2022,January,None,None,,None,None,None,None,None,None,None,None,None,88.0,None,Serbia,United Kingdom of Great Britain and Northern Ireland,None,2.302585,0,0,0,1,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [6]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98057 entries, 0 to 98056
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98057 non-null  object 
 1   name                       98057 non-null  object 
 2   info                       98057 non-null  object 
 3   link                       98057 non-null  object 
 4   num_references             98057 non-null  int64  
 5   year                       98057 non-null  int64  
 6   month                      98057 non-null  object 
 7   info_parenth               36661 non-null  object 
 8   info_1                     22 non-null     object 
 9   info_2                     98025 non-null  object 
 10  info_3                     48897 non-null  object 
 11  info_4                     10264 non-null  object 
 12  info_5                     1265 non-null   object 
 13  info_6                     181 non-null    obj

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for values by rebuilding `known_for_dict`.

### Extracting `known_for` Continued

#### Finding `known_for` Roles in `info_2`

In [47]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [46]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [45]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "translator" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [44]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [48]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "and Bible translator" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [49]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "linguist and bible translator"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [50]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = []
sciences = []

business_farming = []
academia_humanities = [
    "and translator of philosophy and literature",
    "scholar and translator of literature",
    "translator and literature scholar",
    "translator and literary scholar",
    "translator of modern literature",
    "language scholar and translator",
    "linguist and bible translator",
    "medievalist and translator",
    "litterateur and translator",
    "sinologist and translator",
    "translator of literature",
    "translator and linguist",
    "linguist and translator",
    "teacher and translator",
    "scholar and translator",
    "and Bible translator",
    "literary translator",
    "translator and",
    "and translator",
    "translator",
]
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [51]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [53]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['academia_humanities'] ==1].sample(2)

CPU times: total: 10.2 s
Wall time: 10.2 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
86445,3,Oyewusi Ibidapo-Obe,", 71, Nigerian academic administrator, vice chancellor of the University of Lagos , COVID-19.",https://en.wikipedia.org/wiki/Oyewusi_Ibidapo-Obe,15,2021,January,,None,,vice chancellor of the University of Lagos,COVID,None,None,None,None,None,None,None,71.0,None,Nigeria,None,2000 2007,2.772589,0,0,0,1,0,0,0,0,0,0,0,0,1
59476,6,Don Welch,", 84, American poet and academic.",https://en.wikipedia.org/wiki/Don_Welch,3,2016,August,None,None,,None,None,None,None,None,None,None,None,None,84.0,None,United States of America,None,None,1.386294,0,0,0,1,0,1,0,0,0,0,0,0,2


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [54]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 28392 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [237]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [236]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [235]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "film" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [234]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [232]:
# Example code to quick-screen values that may overlap categories
df.loc[[index for index in df.index if "cultural researcher" in df.loc[index, "info"]]]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
74555,30,Robert R. Spitzer,", 96, American agricultural researcher and educator.",https://en.wikipedia.org/wiki/Robert_R._Spitzer,3,2019,April,None,None,agricultural researcher,None,None,None,None,None,None,None,None,None,96.0,None,United States of America,None,None,1.386294,0,0,0,1,0,0,0,0,0,0,0,0,1
93001,26,Kirill Razlogov,", 75, Russian film critic and cultural researcher.",https://en.wikipedia.org/wiki/Kirill_Razlogov,6,2021,September,None,None,film critic and cultural researcher,None,None,None,None,None,None,None,None,None,75.0,None,Russia,None,None,1.945910,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [231]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "censor" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [230]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "film subject and domestic abuse symbol"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [228]:
# Creating lists for each category
politics_govt_law = [
    "censor",
]

arts = [
    "documentary filmmaker and pioneer of public access television",
    "television documentary director and filmmaker",
    "film and television special effects designer",
    "Bollywood filmmaker and brother of Dev Anand",
    "rock tour organiser and film studio manager",
    "film musical arranger musical orchestrator",
    "film and television editor and director",
    "filmmaker and children book illustrator",
    "film critic and film festival director",
    "experimental filmmaker and glass maker",
    "film studio executive and talent agent",
    "film editor and Academy Award winner",
    "film and television costume designer",
    "BBC Northern broadcaster & filmmaker",
    "film critic and television presenter",
    "director and producer in film and TV",
    "television executive and filmmaker",
    "music director for Bollywood films",
    "film studio executive and producer",
    "filmmaker and television producer",
    "wildlife film maker and producer",
    "director for film and television",
    "underwater documentary filmmaker",
    "film documentarian and producer",
    "filmmaker and festival promoter",
    "graphic designer and filmmaker",
    "film and advertising executive",
    "film critic and radio producer",
    "film distributor and producer",
    "ʼNamgis documentary filmmaker",
    "cinematographer and filmmaker",
    "and film and theater director",
    "television and film executive",
    "film and television executive",
    "film and television producer",
    "Arabian film and TV director",
    "film and stage choreographer",
    "stage director and filmmaker",
    "choreographer and filmmaker",
    "music documentary filmmaker",
    "film critic and researcher",
    "independent film executive",
    "film and television editor",
    "columnist and film critic",
    "Oscar winning film editor",
    "and documentary filmmaker",
    "film and theater director",
    "and documentary filmmaker",
    "documentary filmmaker and",
    "film and theatre producer",
    "film and theater producer",
    "film and theatre director",
    "film production designer",
    "music and film executive",
    "producer of horror films",
    "and underwater filmmaker",
    "film critic and essayist",
    "film television producer",
    "film marketing publicist",
    "film critic and producer",
    "film editor and producer",
    "and aerial film operator",
    "film editor and director",
    "film critic and director",
    "film industry executive",
    "film and opera director",
    "theater and film critic",
    "film and stage director",
    "filmmaker and cameraman",
    "Emmy Award winning film",
    "documentary film editor",
    "Tony Award winning film",
    "film and theatre critic",
    "theatre and film critic",
    "documentary film maker",
    "experimental filmmaker",
    "filmmaker and director",
    "animator and filmmaker",
    "film critic for on ABC",
    "independent film maker",
    "pornographic filmmaker",
    "filmmaker and designer",
    "producer and filmmaker",
    "filmmaker and producer",
    "documentary filmmaker",
    "film studio executive",
    "advertising filmmaker",
    "avant garde filmmaker",
    "music and film critic",
    "film costume designer",
    "independent filmmaker",
    "film  television host",
    "film and TV director",
    "surrealist filmmaker",
    "film and TV producer",
    "filmmaker and editor",
    "film camera operator",
    "film music director",
    "film location scout",
    "film stunt director",
    "wildlife filmmaker",
    "film prop designer",
    "film choreographer",
    "film distributor",
    "of film studies",
    "film programmer",
    "South filmmaker",
    "and film critic",
    "adult film star",
    "film editor and",
    "film critic and",
    "film executive",
    "film trumpeter",
    "and filmmaker",
    "filmmaker and",
    "film lyricist",
    "film pioneer",
    "film editor",
    "film critic",
    "film maker",
    "filmmaker",
    "film star",
    "film and",
    "and film",
    "film",
]
sports = []
sciences = [
    "restorer",
    "virtual reality technology pioneer and",
]

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = ["Anglican prelate and theologian", "Anglican prelate"]
social = []
crime = []
event_record_other = [
    "ALD patient portrayed in the film",  # before arts
    "filmgoer",
    "film subject and domestic abuse symbol",
]
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [229]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "sports": sports,
    "arts": arts,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [238]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['spiritual'] ==1].sample(2)

CPU times: total: 1min 35s
Wall time: 1min 35s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
88862,1,Rayappu Joseph,", 80, Sri Lankan Roman Catholic prelate, bishop of Mannar .",https://en.wikipedia.org/wiki/Rayappu_Joseph,11,2021,April,,None,,bishop of Mannar,None,None,None,None,None,None,None,None,80.0,None,Sri Lanka,Italy,1992 2016,2.484907,0,0,1,0,0,0,0,0,0,0,0,0,1
37277,4,Giovanni Volta,", 83, Italian Roman Catholic prelate, Bishop of Pavia .",https://en.wikipedia.org/wiki/Giovanni_Volta,9,2012,February,,None,,Bishop of Pavia,None,None,None,None,None,None,None,None,83.0,None,Italy,Italy,1986 2003,2.302585,0,0,1,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [239]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 27795 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [465]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [464]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [463]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "professor" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [462]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [466]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "and communication professor" in df.loc[index, "info"]
#     ]
# ]

<IPython.core.display.Javascript object>

In [467]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "professor and World War II researcher"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [468]:
# Creating lists for each category
politics_govt_law = [
    "who helped uncover the Bay of Pigs Invasion plan",
    "child protection expert",
]

arts = []
sports = []
sciences = [
    "who made critical contributions to the development of radar",
]

business_farming = []
academia_humanities = [
    "professor and official pronouncer of the Scripps National Spelling Bee from to",
    "geographer and Alexander von Humboldt professor of geography at UCLA",
    "professor and twice interim president of the University of Missouri",
    "professor of and Islamic Studies at the University of Edinburgh",
    "linguistics professor and Pacific Islands language specialist",
    "professor of education and commentator on education topics",
    "professor at Columbia University and scholar of literature",
    "professor of education at the University of Washington",
    "professor and leading researcher into category theory",
    "professor of aesthetics at University of Strasbourg",
    "and drama professor at the Academy of Theatre Arts",
    "professor of Assyriology and Babylonian literature",
    "professor at Princeton Theological Seminary and",
    "professor of history at Indiana University",
    "professor of History at University College",
    "classical scholar and history professor",
    "ist and professor of ancient languages",
    "and professor at Seton Hall University",
    "professor at the University of Chicago",
    "professor and World War II researcher",
    "professor at Brigham Young University",
    "professor and folklorist of cultures",
    "and professor of clinical psychology",
    "professor of comparative literature",
    "scholar and professor of literature",
    "professor specialized in turbulence",
    "and political philosophy professor",
    "professor of modern Jewish history",
    "professor and daughter of Zhu De",
    "professor at Stanford University",
    "professor at Columbia University",
    "professor of Ancient Philosophy",
    "professor of Early Christianity",
    "professor of Jewish literature",
    "professor of Hebrew Literature",
    "and professor at University of",
    "professor at the University of",
    "Stanford University professor",
    "professor emeritus of history",
    "professor of at University of",
    "anthropologist and professor",
    "professor of and runologist",
    "and communication professor",
    "and professor of philosophy",
    "researcher and professor of",
    "Assyriologist and professor",
    "emeritus professor at Yale",
    "professor at University of",
    "professor of Asian studies",
    "professor of philosophy of",
    "professor of women studies",
    "library science professor",
    "ethnologist and professor",
    "folklorist and professor",
    "and philosophy professor",
    "geographer and professor",
    "A&M University professor",
    "professor and sinologist",
    "and University professor",
    "and university professor",
    "librarian and professor",
    "University of professor",
    "professor of Egyptology",
    "women studies professor",
    "pedagogue and professor",
    "professor of philosophy",
    "professor of literature",
    "linguist and professor",
    "professor of geography",
    "and Emeritus professor",
    "and professor emeritus",
    "assistant professor of",
    "professor of Classics",
    "professor emeritus of",
    "scholar and professor",
    "professor of rhetoric",
    "professor of classics",
    "and college professor",
    "and professor of law",
    "literature professor",
    "professor of Studies",
    "professor of studies",
    "professor of history",
    "philosophy professor",
    "university professor",
    "professor of Hebrew",
    "associate professor",
    "professor emeritus",
    "language professor",
    "and law professor",
    "law professor and",
    "college professor",
    "history professor",
    "and a professor",
    "law professor",
    "MIT professor",
    "and professor",
    "professor and",
    "professor of",
    "professor in",
    "professor",
]
law_enf_military_operator = []
spiritual = [
    "expert on biblical manuscripts",
]
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [469]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [470]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['academia_humanities'] ==1].sample(2)

CPU times: total: 1min 13s
Wall time: 1min 13s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
82560,27,David Stronach,", 89, Scottish archaeologist.",https://en.wikipedia.org/wiki/David_Stronach,5,2020,June,None,None,,None,None,None,None,None,None,None,None,None,89.0,None,Scotland,None,None,1.791759,0,0,0,1,0,0,0,0,0,0,0,0,1
48614,21,Tritobia Hayes Benjamin,", 69, American art historian and educator.",https://en.wikipedia.org/wiki/Tritobia_Hayes_Benjamin,6,2014,June,None,None,,None,None,None,None,None,None,None,None,None,69.0,None,United States of America,None,None,1.945910,0,0,0,1,0,1,0,0,0,0,0,0,2


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [471]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 27546 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [564]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [563]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [562]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "theologian" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [561]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [558]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "founder of magazine" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [557]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "liberation theologian"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [554]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = [
    "evangelical Protestant pastor and theologian",
    "Christian Protestant Ecumenical theologian",
    "congregationalist minister and theologian",
    "evangelical theologian and missiologist",
    "Eastern Orthodox priest and theologian",
    "Catholic Jesuit priest and theologian",
    "Presbyterian minister and theologian",
    "theologian and Catholic lay leader",
    "Protestant theologian and biblical",
    "Anglican clergyman and theologian",
    "Evangelical Christian theologian",
    "theologian and Dead Sea Scrolls",
    "Franciscan friar and theologian",
    "theologian and religious leader",
    "Baptist minister and theologian",
    "Anglican priest and theologian",
    "Anglican bishop and theologian",
    "Lutheran theologian and bishop",
    "theologian and Bishop of Medak",
    "Catholic priest and theologian",
    "Jesuit priest and theologian",
    "theologian and Old Testament",
    "theologian and New Testament",
    "dispensationalist theologian",
    "Southern Baptist theologian",
    "Eastern Orthodox theologian",
    "Evangelical theologian and",
    "missionary and theologian",
    "theologian and missionary",
    "theologian and ecumenist",
    "minister and theologian",
    "chaplain and theologian",
    "theologian and biblical",
    "theologian and exegete",
    "evangelical theologian",
    "priest and theologian",
    "Jesuit theologian and",
    "liberation theologian",
    "bishop and theologian",
    "theologian and pastor",
    "theologian and priest",
    "protestant theologian",
    "cleric and theologian",
    "Protestant theologian",
    "Methodist theologian",
    "Church of theologian",
    "Christian theologian",
    "Catholic theologian",
    "Lutheran theologian",
    "Qur'anic theologian",
    "Anglican theologian",
    "Islamic theologian",
    "Jewish theologian",
    "Jesuit theologian",
    "Sunni theologian",
    "Queer theologian",
    "lay theologian",
    "and theologian",
    "theologian and",
    "theologian",
]
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [565]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [566]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['spiritual'] ==1].sample(2)

CPU times: total: 45.6 s
Wall time: 45.6 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
39700,27,Tony Dumper,", 88, British Anglican prelate, Bishop of Dudley .",https://en.wikipedia.org/wiki/Tony_Dumper,13,2012,August,,None,,Bishop of Dudley,None,None,None,None,None,None,None,None,88.0,None,United Kingdom of Great Britain and Northern Ireland,None,1977 1993,2.639057,0,0,1,0,0,0,0,0,0,0,0,0,1
75122,4,Geevarghese Mar Timotheos,", 91, Indian Eastern Catholic prelate, Bishop of Tiruvalla .",https://en.wikipedia.org/wiki/Geevarghese_Mar_Timotheos,3,2019,June,,None,,Bishop of Tiruvalla,None,None,None,None,None,None,None,None,91.0,None,India,None,1988 2003,1.386294,0,0,1,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [567]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 27340 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [635]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [634]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [633]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "linguist" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [631]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [632]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "computational" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [636]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "psycholinguist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [637]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = []
sciences = [
    "neuro linguistic programming expert",  # before academia_humanities
    "psycholinguist",
]

business_farming = []
academia_humanities = [
    "linguist and leading scholar of Mon and Khmer languages",
    "linguist and classical scholar who deciphered Linear B",
    "sociolinguist and linguistic anthropologist",
    "linguistic anthropologist and semiotician",
    "linguist specialized in Romance languages",
    "scholar of literature and linguistics",
    "Yiddish linguist and lexicographer",
    "linguist and literature scholar",
    "linguist and anthropologist",
    "anthropologist and linguist",
    "linguist and Hittitologist",
    "lexicographer and linguist",
    "linguist and lexicographer",
    "musicologist and linguist",
    "linguistic anthropologist",
    "albanologist and linguist",
    "linguist and ethnologist",
    "philologist and linguist",
    "linguist and philologist",
    "linguist and celtologist",
    "linguist and Iranologist",
    "grammarian and linguist",
    "sinologist and linguist",
    "linguistics expert and",
    "Santhali linguist and",
    "scholar and linguist",
    "linguist and scholar",
    "linguist and slavist",
    "linguist and teacher",
    "linguist of descent",
    "linguistics scholar",
    "historical linguist",
    "linguistics expert",
    "classical linguist",
    "Creole linguist",
    "sociolinguist",
    "and linguist",
    "linguist and",
    "linguist",
]
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [638]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
    "academia_humanities": academia_humanities,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [639]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['academia_humanities'] ==1].sample(2)

CPU times: total: 27.5 s
Wall time: 27.5 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
22023,11,Joachim Fest,", 79, German historian and journalist.",https://en.wikipedia.org/wiki/Joachim_Fest,11,2006,September,None,None,,None,None,None,None,None,None,None,None,None,79.0,None,Germany,None,None,2.484907,0,0,0,1,0,1,0,0,0,0,0,0,2
58774,19,Mihnea Berindei,", 68, Romanian-born French historian.",https://en.wikipedia.org/wiki/Mihnea_Berindei,3,2016,June,None,None,,None,None,None,None,None,None,None,None,None,68.0,None,Romania,France,None,1.386294,0,0,0,1,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [640]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 27147 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [739]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [738]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [737]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "anthropologist" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [736]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [734]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "griot" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [733]:
# # Example code to quick-checouk a specific entry
# df[df["info_2"] == "physical anthropologist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [731]:
# Creating lists for each category
politics_govt_law = []

arts = [
    "griot",
]
sports = []
sciences = [
    "palaeontologist and palaeoanthropologist",
    "geologist and paleoanthropologist",
    "forensic anthropologist",
    "physical anthropologist",
    "palaeoanthropologist",  # before academia_humanities
    "paleoanthropologist",
]

business_farming = [
    "banker and member of the Rothschild family",
    "investment banker and financier",
    "beverage executive and banker",
    "banker and venture capitalist",
    "Arabian billionaire banker",
    "investment banker and",
    "financier and banker",
    "banker and executive",
    "banker and chairman",
    "merchant banker and",
    "and merchant banker",
    "billionaire banker",
    "investment banker",
    "mortgage banker",
    "merchant banker",
    "banker and",
    "and banker",
    "banker",
]
academia_humanities = [
    "anthropologist specializing in Aztec culture",
    "social anthropologist ands ethnographer",
    "social anthropologist and musicologist",
    "anthropologist and cryptozoologist",
    "anthropologist and museum director",
    "anthropologist and ethnographer",
    "ethnologist and anthropologist",
    "indigenous Hopi anthropologist",
    "Africanist and anthropologist",
    "anthropologist and folklorist",
    "anthropologist and ethicist",
    "anthropologist and scholar",
    "social anthropologist and",
    "culinary anthropologist",
    "cultural anthropologist",
    "social anthropologist",
    "anthropologist and",
    "and anthropologist",
    "anthropologist",
]
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [732]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
    "sciences": sciences,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [741]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['business_farming'] ==1].sample(2)

CPU times: total: 23.7 s
Wall time: 23.7 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
30873,18,Jerry York,", 71, American businessman, Executive Director of Apple Inc., cerebral hemorrhage.",https://en.wikipedia.org/wiki/Jerry_York_(businessman),8,2010,March,None,None,,Executive Director of Apple Inc,cerebral hemorrhage,None,None,None,None,None,None,None,71.0,None,United States of America,None,None,2.197225,0,0,0,0,1,0,0,0,0,0,0,0,1
71189,11,Sir Doug Ellis,", 94, English entrepreneur and football club chairman .",https://en.wikipedia.org/wiki/Doug_Ellis,27,2018,October,Aston Villa,None,football club chairman,None,None,None,None,None,None,None,None,None,94.0,None,United Kingdom of Great Britain and Northern Ireland,None,Aston Villa,3.332205,0,0,0,0,1,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [742]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 26828 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [763]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [762]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [747]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "racing cyclist" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [1346]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [764]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "racing cyclist and manager"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [765]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [
    "track and road racing cyclist",
    "triathlete and racing cyclist",
    "Olympic road racing cyclist",
    "professional racing cyclist",
    "racing cyclist and manager",
    "Paralympic racing cyclist",
    "racing cyclist and sports",
    "Olympic racing cyclist",
    "road racing cyclist",
    "racing cyclist and",
    "and racing cyclist",
    "racing cyclist",
]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [766]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [767]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sports'] ==1].sample(2)

CPU times: total: 6.14 s
Wall time: 6.14 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
95772,31,Mike Nykoluk,", 87, Canadian ice hockey player and coach .",https://en.wikipedia.org/wiki/Mike_Nykoluk,13,2022,January,"Toronto Maple Leafs, Hershey Bears",None,,None,None,None,None,None,None,None,None,None,87.0,None,Canada,None,"Toronto Maple Leafs, Hershey Bears",2.639057,0,0,0,0,0,0,1,0,0,0,0,0,1
83014,19,Jack McIlhargey,", 68, Canadian ice hockey player and coach , cancer.",https://en.wikipedia.org/wiki/Jack_McIlhargey,12,2020,July,"Vancouver Canucks, Philadelphia Flyers, Hartford Whalers",None,,cancer,None,None,None,None,None,None,None,None,68.0,None,Canada,None,"Vancouver Canucks, Philadelphia Flyers, Hartford Whalers",2.564949,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [768]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 26640 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [923]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [922]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [921]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "sports" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [919]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [920]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "car builder" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [917]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "motorsports manager and car builder"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [915]:
# Creating lists for each category
politics_govt_law = []

arts = [
    "publicist for the University of Maryland and the Washington Redskins",
    "marketing executive and sports imposter",
    "fashion designer and sportswear pioneer",
    "sportswear and fashion designer",  # before sports
]
sports = [
    "IOC sports administrator and Olympic sport shooter",
    "hunting and fishing specialist and outdoor sports",
    "Olympic gold medal winning wrestler and sports",
    "sportsman and college athletics administrator",
    "Olympic silver medal winning sports shooter",
    "athletics coach and sports administrator",
    "netball player and sports administrator",
    "sports shooter and Olympic champion",
    "Olympic sprinter and sports coach",
    "athlete and sports administrator",
    "fencer and sports administrator",
    "sports baseball official scorer",
    "ski jumper and sports official",
    "triple international sportsman",
    "professional sports team owner",
    "basketball referee and sports",
    "orienteer and sports official",
    "football executive and sports",
    "Hall of Fame sports executive",
    "college sports administrator",
    "Baseball Hall of Fame sports",
    "Negro league baseball sports",
    "cricket player and sportsman",
    "cyclist and sports director",
    "fencer and sports executive",
    "Olympic sports commissioner",
    "and Olympic sports shooter",
    "wheelchair sports athlete",
    "sports franchise co owner",
    "high school sports coach",
    "and sports administrator",
    "sports administrator and",
    "athlete and sports coach",
    "weightlifter and sports",
    "sports  sport executive",
    "sports player and coach",
    "motorsports manager and",
    "Olympic sports shooter",
    "and sports team owner",
    "sports administrator",
    "and sports executive",
    "motorsports director",
    "and sports official",
    "Hall of Fame sports",
    "sports shooter and",
    "sports manager and",
    "and sports shooter",
    "sports team owner",
    "golfer and sports",
    "sports club owner",
    "sports coach and",
    "sports executive",
    "cricket  sports",
    "baseball sports",
    "and sportswoman",
    "Republic sports",
    "sportswoman and",
    "sports director",
    "sports promoter",
    "sports official",
    "sports shooter",
    "sportsman and",
    "sportsperson",
    "sports diver",
    "sports coach",
    "sports agent",
    "motorsports",
    "sportsman",
    "and sports",
    "sports and",
    "sports",
]
sciences = []

business_farming = [
    "chairman and owner of Ellen Tracy sportswear",  # before sports
]
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = [
    "thoroughbred racehorse involved in sports betting substitution scandal",  # before sports
]

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [916]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [924]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sports'] ==1].sample(2)

CPU times: total: 38.8 s
Wall time: 38.8 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
76618,9,Neiron Ball,", 27, American football player , arteriovenous malformation.",https://en.wikipedia.org/wiki/Neiron_Ball,5,2019,September,Oakland Raiders,None,,arteriovenous malformation,None,None,None,None,None,None,None,None,27.0,None,United States of America,None,Oakland Raiders,1.791759,0,0,0,0,0,0,1,0,0,0,0,0,1
52173,25,Hannes Baldauf,", 76, German football player and coach.",https://en.wikipedia.org/wiki/Hannes_Baldauf,3,2015,February,None,None,,None,None,None,None,None,None,None,None,None,76.0,None,Germany,None,None,1.386294,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [925]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 26306 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [1100]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [1099]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [1098]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "broadcaster" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [1097]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [1095]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [index for index in df.index if "scientific divulgator" in df.loc[index, "info"]]
# ]

<IPython.core.display.Javascript object>

In [1094]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "broadcaster and secretary to Joseph Goebbels"]

<IPython.core.display.Javascript object>

In [976]:
# Dropping entry with link that points to husband's page
index = df[df["link"] == "https://en.wikipedia.org/wiki/Ramona_Bell"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [1092]:
# Creating lists for each category
politics_govt_law = [
    "secretary to Joseph Goebbels",
]

arts = [
    "button accordion player and radio and television broadcaster",
    "broadcaster and radio and television administrator",
    "BBC broadcaster and transatlantic commentator",
    "first regular broadcaster on CBC Television",
    "news broadcaster and television presenter",
    "broadcaster and public address announcer",
    "and broadcaster for the Cincinnati Reds",
    "broadcaster and cultural administrator",
    "radio broadcaster and television host",
    "broadcaster and television executive",
    "radio broadcaster and documentarian",
    "radio broadcaster and food critic",
    "radio and television broadcaster",
    "newsreader and radio broadcaster",
    "television and radio broadcaster",
    "broadcaster and theatre producer",
    "newspaper editor and broadcaster",
    "broadcaster and television host",
    "radio broadcaster and announcer",
    "radio broadcaster and executive",
    "broadcaster and music arranger",
    "correspondent and broadcaster",
    "music critic and broadcaster",
    "Papua New radio broadcaster",
    "news reader and broadcaster",
    "broadcaster and television",
    "biographer and broadcaster",
    "and television broadcaster",
    "TV and radio broadcaster",
    "Hall of Fame broadcaster",
    "pirate radio broadcaster",
    "and broadcaster known as",
    "broadcaster and theatre",
    "television broadcaster",
    "broadcaster and anchor",
    "cultural administrator",
    "radio news broadcaster",
    "BBC radio broadcaster",
    "and radio broadcaster",
    "radio broadcaster and",
    "outdoors broadcaster",
    "broadcaster for CBC",
    "broadcaster and CEO",
    "radio  broadcaster",
    "radio broadcaster",
    "Māori broadcaster",
    "news broadcaster",
    "BBC broadcaster",
    "broadcaster for",
    "broadcaster and",
    "and broadcaster",
    "broadcaster",
]
sports = [
    "for the Pittsburgh Steelers",
    "yo yo world champion",
]
sciences = [
    "scientific divulgator",
]

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [1093]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [1101]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

CPU times: total: 28.4 s
Wall time: 28.4 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
73141,9,Niki Goulandris,", 94, Greek philanthropist and painter.",https://en.wikipedia.org/wiki/Niki_Goulandris,7,2019,February,None,None,,None,None,None,None,None,None,None,None,None,94.0,None,Greece,None,None,2.079442,0,1,0,0,0,1,0,0,0,0,0,0,2
6291,7,Derek Taylor,", 65, English journalist, writer and record producer, throat cancer.",https://en.wikipedia.org/wiki/Derek_Taylor,33,1997,September,None,None,,writer and record producer,throat cancer,None,None,None,None,None,None,None,65.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.526361,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [1102]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 26099 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [1345]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [1344]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [1343]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "scholar" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [1342]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [1335]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "biblical scholar" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [1340]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "feminist scholar"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [1338]:
# Creating lists for each category
politics_govt_law = [
    "advocate for education reform",
]

arts = []
sports = []
sciences = [
    "Ayurvedic",
    "mass communications",
]

business_farming = []
academia_humanities = [
    "literary scholar and founder of Reader response criticism",
    "and scholar in medieval studies and palaeography",
    "classical scholar who specialized in mythology",
    "scholar known for writings on the Iroquois",
    "philologist and religious studies scholar",
    "librarian and scholar of library science",
    "literary scholar of Native literature",
    "musicologist and Shakespeare scholar",
    "and folklorist and literary scholar",
    "literary scholar and media theorist",
    "and folklorist and literary scholar",
    "media scholar and cultural theorist",
    "scholar and historical revisionist",
    "museum director and Judaic scholar",
    "literary scholar and social critic",
    "language scholar and lexicographer",
    "scholar of the Caucasian cultures",
    "scholar of continental philosophy",
    "literary scholar and medievalist",
    "folklorist and literary scholar",
    "scholar of renaissance humanism",
    "literary scholar and redologist",
    "librarian and Tolkien scholar",
    "and Dead Sea Scrolls scholar",
    "scholar of Semitic languages",
    "scholar in Buddhist studies",
    "Sufism prelate and scholar",
    "scholar and educationalist",
    "scholar of Asian languages",
    "disability studies scholar",
    "dance scholar and curator",
    "and New Testament scholar",
    "communication scholar and",
    "scholar of gender studies",
    "scholar of historiography",
    "scholar and educationist",
    "scholar and bioethicist",
    "orientalist and scholar",
    "scholar and Perak mufti",
    "scholar of ancient law",
    "and scholar of Judaism",
    "scholar and specialist",
    "gender studies scholar",
    "researcher and scholar",
    "scholar of literature",
    "Shakespearean scholar",
    "and theology scholar",
    "scholar and preacher",
    "literary scholar and",
    "and Napoleon scholar",
    "and literary scholar",
    "biblical scholar and",
    "and biblical scholar",
    "and Biblical scholar",
    "medievalist scholar",
    "Tlingit scholar and",
    "scholar of medieval",
    "Chaucer scholar and",
    "scholar and curator",
    "Renaissance scholar",
    "scholar of history",
    "folk music scholar",
    "scholar of studies",
    "manuscript scholar",
    'literature scholar',
    "Torah scholar and",
    "rare book scholar",
    "Holocaust scholar",
    "holocaust scholar",
    "education scholar",
    'classical scholar',
    "Mayanist scholar",
    "language scholar",
    "medieval scholar",
    "scholar and Sufi",
    "oriental scholar",
    "Semitics scholar",
    "religion scholar",
    "classics scholar",
    "literary scholar",
    'Sanskrit scholar',
    "theatre scholar",
    "library scholar",
    "Judaica scholar",
    "Yolngu scholar",
    "comics scholar",
    "Bible scholar",
    "Saxon scholar",
    "Urdu scholar",
    "scholar of",
    "and scholar",
    "scholar and",
    "scholar",
]
law_enf_military_operator = []
spiritual = [
    "Jainist and Buddhist",
    "talmudic",
    "Talmudic",
    "Talmud",
    "Salafi",
    "hadith",
    "Vedic",
]
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [1339]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['academia_humanities'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

In [1103]:
print("dunzo!")

# Sound notification when cell executes
chime.success()

dunzo!


<IPython.core.display.Javascript object>

#### Finding `known_for` Roles in `info_2`

In [ ]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
roles_list.pop()

In [ ]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "general" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

In [ ]:
# # Code to check each specific value
# specific_roles_list.pop()

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "and science writer" in df.loc[index, "info"]]]

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [ ]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "outlaw country music singer songwriter"]

#### Creating Lists for Each `known_for` Category

In [ ]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [


]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Observations:
- It is time to export our dataframe and start a new notebook.

### Exporting Dataset to SQLite Database [wp_life_expect_clean10.db]()

In [ ]:
# # Exporting dataframe

# # Saving dataset in a SQLite database
# conn = sql.connect("wp_life_expect_clean10.db")
# df.to_sql("wp_life_expect_clean10", conn, index=False)

# # Chime notification when cell executes
# chime.success()

# [Proceed to Data Cleaning Part 11]()